## Import packages

In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import time
import pandas as pd
import numpy as np

## Selenium scraper (DON'T USE, USE SEGMENTED CODE BELOW)

#### Change LOCATION, JOBS, and PAGES variables as needed

In [2]:
LOCATIONS = ['Seattle, WA']
LOCATIONS2 = ['Washington DC', 'Denver, CO', 'Chicago, IL', 'San Diego, CA', 
             'Portland, OR', 'Boston, MA', 'Dallas, TX', 'Phoenix, AZ']
JOBS = ['data science']
PAGES = 5

titles = []
companies = []
locations = []
links = []
reviews = []
salaries = []
descriptions = []

#### Intialize driver and scrape 
Runs in headless mode. Loops through all locations and jobs and appends fields to corresponding arrays.

In [27]:
# Driver initialization

options = Options()
options.headless = False
driver = webdriver.Chrome(options=options)
driver.get('https://indeed.com')

initial_search_button = driver.find_element_by_xpath('//*[@id="whatWhereFormId"]/div[3]/button')
initial_search_button.click()


In [ ]:
for l in LOCATIONS:
    for j in JOBS:
        
        starttime = time.time()
        print('Searching {} roles in {}'.format(j, l))
        
        
        advanced_search = driver.find_element_by_xpath("//a[contains(text(),'Advanced Job Search')]")
        advanced_search.click()

        #driver.implicitly_wait(3)
        search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
        search_job.clear()
        search_job.send_keys([j])

        search_loc = driver.find_element_by_xpath('//*[@id="where"]')
        search_loc.clear()
        search_loc.send_keys([l])

        display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="30"]')
        display_limit.click()
        sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
        sort_option.click()
        search_button = driver.find_element_by_xpath('//*[@id="fj"]')
        search_button.click()
        
        try:
            close_popup = driver.find_element_by_xpath('//*[@id="popover-x"]')
            close_popup.click()
        except: 
            pass
            
        driver.implicitly_wait(2)
        driver.refresh()
        
        
        
        for i in range(0, PAGES):
            job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')

            for job in job_card:
                try:
                    review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
                except:
                    review = "None"
                reviews.append(review)

                try:
                    salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
                except:
                    salary = "None"    
                salaries.append(salary)

                try:
                    title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
                except:
                    title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
                titles.append(title)

                links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))

                companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)

                # set location to parameter passed in function call
                locations.append(l)

            try:
                next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
                next_page.click()
            except:
                next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
                next_page.click()
        
        
        
        endtime = time.time()
        process_time = round((endtime-starttime)/60,2)
        print("Processing Time: {} min".format(process_time))



## Segmented Script
Prevent errors in script

In [11]:
LOCATION = 'Seattle, WA'
JOB = 'data science'
PAGES = 5

titles = []
companies = []
locations = []
links = []
reviews = []
salaries = []
descriptions = []

In [6]:

try:
    close_popup = driver.find_element_by_xpath('//*[@id="popover-x"]')
    close_popup.click()
except: 
    pass

In [9]:
advanced_search = driver.find_element_by_xpath("//a[contains(text(),'Advanced Job Search')]")
advanced_search.click()

In [12]:
search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
search_job.clear()
search_job.send_keys([JOB])

search_loc = driver.find_element_by_xpath('//*[@id="where"]')
search_loc.clear()
search_loc.send_keys([LOCATION])

display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="30"]')
display_limit.click()
sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
sort_option.click()
search_button = driver.find_element_by_xpath('//*[@id="fj"]')
search_button.click()

In [14]:
for i in range(0, PAGES):
    job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')

    for job in job_card:
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)

        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"    
        salaries.append(salary)

        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)

        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))

        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)

        # set location to parameter passed in function call
        locations.append(LOCATION)

    try:
        next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()
    except:
        next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        next_page.click()

#### Loop through URLs to get job descriptions

In [29]:
descriptions=[]
for link in links:
    driver.get(link)
    try:
        jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    except:
        jd = 'None'
    jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)

In [30]:
driver.quit()

#### Save results to csv 

In [38]:
locations = locations.append(LOCATION)
df_output = pd.DataFrame()
df_output['Title'] = titles
df_output['Company'] = companies
df_output['Location'] = locations
df_output['Link'] = links
df_output['Review'] = reviews
df_output['Salary'] = salaries
df_output['Description'] = descriptions

df_output.to_csv('./data/selenium/'+ LOCATION[:LOCATION.find(',')].lower() +'.csv')

## Concatenate all csvs together

In [45]:
# Find all csvs 
import os, fnmatch

def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

filenames = find('*.csv', './data/selenium/')

In [46]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in filenames])
#export to csv
combined_csv.to_csv('./data/selenium/all_jobs.csv', index=False, encoding='utf-8-sig')

In [47]:
df_all = pd.read_csv('./data/selenium/all_jobs.csv', encoding='utf-8-sig')
df_all.head()


,Unnamed: 0,Title,Company,Location,Link,Review,Salary,Description
0,0,Machine Learning Engineer - Intern,NXP Semiconductors,"San Francisco, CA",https://www.indeed.com/rc/clk?jk=eba29634073e4...,3.8,None,NXP Semiconductors N.V. (NASDAQ: NXPI) enables...
1,1,Machine Learning Compiler Optimization Enginee...,"Advanced Micro Devices, Inc.","San Francisco, CA",https://www.indeed.com/rc/clk?jk=eeb68b20aca75...,3.8,None,"What you do at AMD changes everything\nAt AMD,..."
2,2,Senior Machine Learning Engineer - Activision ...,King.com,"San Francisco, CA",https://www.indeed.com/rc/clk?jk=8b00dc341998e...,None,None,Craft:\nTechnology & Development\nJob Descript...
3,3,Data Engineer - Opportunity for Working Remote...,VMware,"San Francisco, CA",https://www.indeed.com/rc/clk?jk=6477612882753...,4.0,None,The Elevator Pitch: Why will you enjoy this ne...
4,4,"Director, Risk Management Technology–Data Office",KPMG,"San Francisco, CA",https://www.indeed.com/rc/clk?jk=4910e796eb73c...,4.0,None,Known for being a great place to work and buil...


In [52]:
df_all['Salary'] =df_all['Salary'].replace('None', np.nan)
df_salaries = df_all.dropna(axis=0, subset=['Salary'])


49

In [53]:
df_salaries.to_csv('./data/selenium/jobs_with_salaries.csv', index=False, encoding='utf-8-sig')